In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import seaborn as sns
from tabulate import tabulate
import os

# Create Image directory if it doesn't exist
if not os.path.exists('Image'):
    os.makedirs('Image')

# Set seed untuk reproducibility
np.random.seed(42)

# Parameter simulasi
lambda_rate = 5  # Rata-rata jumlah barang cacat per hari
num_days = 30     # Jumlah hari simulasi (iterasi)
num_simulations = 5  # Jumlah simulasi berbeda yang akan dijalankan

def run_simulation(lambda_rate, num_days):
    """
    Menjalankan simulasi jumlah barang cacat selama num_days hari
    dengan rata-rata lambda_rate per hari.
    """
    # Menghasilkan data acak dari distribusi Poisson
    defect_counts = np.random.poisson(lambda_rate, num_days)
    return defect_counts

def calculate_statistics(data):
    """
    Menghitung statistik deskriptif dari data.
    """
    mean = np.mean(data)
    variance = np.var(data)
    std_dev = np.std(data)
    minimum = np.min(data)
    maximum = np.max(data)
    median = np.median(data)
    
    # Menghitung probabilitas tidak ada cacat (0 cacat)
    prob_zero = np.sum(data == 0) / len(data)
    
    return {
        "Mean": mean,
        "Variance": variance,
        "Std Deviation": std_dev,
        "Minimum": minimum,
        "Maximum": maximum,
        "Median": median,
        "P(X=0)": prob_zero
    }

def theoretical_values(lambda_rate):
    """
    Menghitung nilai teoritis distribusi Poisson.
    """
    mean = lambda_rate
    variance = lambda_rate
    std_dev = np.sqrt(lambda_rate)
    prob_zero = np.exp(-lambda_rate)
    
    return {
        "Mean": mean,
        "Variance": variance,
        "Std Deviation": std_dev,
        "P(X=0)": prob_zero
    }

# 1. Menjalankan multiple simulasi
all_simulations = []
simulation_stats = []

for i in range(num_simulations):
    # Variasi lambda untuk simulasi berbeda
    sim_lambda = lambda_rate * (0.8 + 0.1 * i)
    
    # Menjalankan simulasi
    defect_counts = run_simulation(sim_lambda, num_days)
    all_simulations.append(defect_counts)
    
    # Menghitung statistik
    stats_sim = calculate_statistics(defect_counts)
    stats_sim["Simulation"] = f"Sim {i+1} (λ={sim_lambda:.1f})"
    stats_sim["Lambda"] = sim_lambda
    simulation_stats.append(stats_sim)

# 2. Histogram semua simulasi sebagai gambar terpisah
plt.figure(figsize=(10, 6))
for i, sim_data in enumerate(all_simulations):
    sns.histplot(sim_data, kde=True, stat="density", alpha=0.5, 
                 label=f"Sim {i+1} (λ={simulation_stats[i]['Lambda']:.1f})")

# Menambahkan kurva PDF teoretis untuk perbandingan
x = np.arange(0, max([max(sim) for sim in all_simulations]) + 5)
for i, stats_dict in enumerate(simulation_stats):
    lambda_val = stats_dict["Lambda"]
    plt.plot(x, stats.poisson.pmf(x, lambda_val), 'o--', 
             label=f"Teoretis λ={lambda_val:.1f}", alpha=0.7, markersize=4)

plt.title("Distribusi Jumlah Barang Cacat per Hari")
plt.xlabel("Jumlah Barang Cacat")
plt.ylabel("Probabilitas")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
# Save histogram as separate image
plt.savefig('Image/distribusi_jumlah_barang_cacat.png', dpi=300, bbox_inches='tight')
plt.close()

# 3. Visualisasi deret waktu dari semua simulasi sebagai gambar terpisah
plt.figure(figsize=(10, 6))
for i, sim_data in enumerate(all_simulations):
    plt.plot(range(1, num_days+1), sim_data, '-o', alpha=0.7, 
             label=f"Sim {i+1} (λ={simulation_stats[i]['Lambda']:.1f})")
    
plt.axhline(y=lambda_rate, color='r', linestyle='--', label=f'Teoretis λ={lambda_rate}')
plt.title("Tren Jumlah Barang Cacat per Hari")
plt.xlabel("Hari ke-")
plt.ylabel("Jumlah Barang Cacat")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
# Save time series as separate image
plt.savefig('Image/tren_jumlah_barang_cacat.png', dpi=300, bbox_inches='tight')
plt.close()

# 4. Boxplot perbandingan distribusi sebagai gambar terpisah
plt.figure(figsize=(10, 6))
boxplot_data = pd.DataFrame(all_simulations).T
boxplot_data.columns = [f"Sim {i+1} (λ={simulation_stats[i]['Lambda']:.1f})" for i in range(num_simulations)]
sns.boxplot(data=boxplot_data)
plt.title("Perbandingan Distribusi Jumlah Barang Cacat")
plt.xlabel("Simulasi")
plt.ylabel("Jumlah Barang Cacat")
plt.xticks(rotation=45)
plt.grid(alpha=0.3)
plt.tight_layout()
# Save boxplot as separate image
plt.savefig('Image/perbandingan_distribusi_boxplot.png', dpi=300, bbox_inches='tight')
plt.close()

# 5. Heatmap jumlah cacat berdasarkan hari dan simulasi sebagai gambar terpisah
plt.figure(figsize=(10, 8))
heatmap_data = pd.DataFrame(all_simulations).T
heatmap_data.columns = [f"Sim {i+1}" for i in range(num_simulations)]
heatmap_data.index = [f"Hari {i+1}" for i in range(num_days)]
sns.heatmap(heatmap_data, cmap="YlOrRd", annot=True, fmt="d", cbar_kws={'label': 'Jumlah Barang Cacat'})
plt.title("Heatmap Jumlah Barang Cacat per Hari")
plt.tight_layout()
# Save heatmap as separate image
plt.savefig('Image/heatmap_jumlah_barang_cacat.png', dpi=300, bbox_inches='tight')
plt.close()

# 6. Perbandingan statistik hasil simulasi dengan nilai teoritis
print("===== Perbandingan Statistik Hasil Simulasi dengan Nilai Teoritis =====")
comparison_data = []

for stats_dict in simulation_stats:
    lambda_val = stats_dict["Lambda"]
    theo_values = theoretical_values(lambda_val)
    
    row = [
        stats_dict["Simulation"],
        f"{stats_dict['Mean']:.2f} / {theo_values['Mean']:.2f}",
        f"{stats_dict['Variance']:.2f} / {theo_values['Variance']:.2f}",
        f"{stats_dict['Std Deviation']:.2f} / {theo_values['Std Deviation']:.2f}",
        f"{stats_dict['P(X=0)']:.4f} / {theo_values['P(X=0)']:.4f}",
    ]
    comparison_data.append(row)

headers = ["Simulasi", "Mean (Sim/Teoritis)", "Variance (Sim/Teoritis)", 
           "Std Dev (Sim/Teoritis)", "P(X=0) (Sim/Teoritis)"]
print(tabulate(comparison_data, headers=headers, tablefmt="grid"))

# 7. Menghitung nilai Goodness of Fit (Chi-square)
print("\n===== Goodness of Fit Test (Chi-Square) =====")
chi_square_results = []

for i, sim_data in enumerate(all_simulations):
    lambda_val = simulation_stats[i]["Lambda"]
    
    # Menghitung frekuensi yang diamati
    observed_freq = np.bincount(sim_data)
    
    # Menghitung frekuensi yang diharapkan
    max_x = max(15, np.max(sim_data) + 1)  # Batasi hingga maksimal nilai atau 15
    expected_probs = stats.poisson.pmf(np.arange(max_x), lambda_val)
    expected_freq = expected_probs * len(sim_data)
    
    # Pastikan panjang array sama
    if len(observed_freq) < max_x:
        observed_freq = np.pad(observed_freq, (0, max_x - len(observed_freq)), 'constant')
    elif len(observed_freq) > max_x:
        expected_freq = np.pad(expected_freq, (0, len(observed_freq) - max_x), 'constant')
    
    # Gabungkan kategori untuk expected_freq < 5
    # (ini adalah penyederhanaan, dalam praktiknya mungkin perlu pendekatan lebih detil)
    valid_indices = expected_freq >= 5
    chi_square = np.sum(((observed_freq[valid_indices] - expected_freq[valid_indices]) ** 2) / 
                        expected_freq[valid_indices])
    
    # Derajat kebebasan (jumlah kategori - 1 - jumlah parameter yang diestimasi)
    # Dalam kasus ini, kita mengestimasi 1 parameter (lambda)
    df = np.sum(valid_indices) - 1 - 1
    
    # Probabilitas p-value
    if df > 0:
        p_value = 1 - stats.chi2.cdf(chi_square, df)
        result = "Tidak ditolak" if p_value > 0.05 else "Ditolak"
    else:
        p_value = None
        result = "Tidak dapat dihitung (df <= 0)"
    
    chi_square_results.append([
        f"Sim {i+1} (λ={lambda_val:.1f})",
        f"{chi_square:.2f}",
        f"{df}",
        f"{p_value:.4f}" if p_value is not None else "N/A",
        result
    ])

chi_headers = ["Simulasi", "Chi-Square", "df", "p-value", "H0 (α=0.05)"]
print(tabulate(chi_square_results, headers=chi_headers, tablefmt="grid"))

# 8. Analisis Perbandingan dengan Distribusi Lain
print("\n===== Perbandingan dengan Distribusi Lain =====")

# Menggunakan data dari simulasi pertama
sample_data = all_simulations[0]
lambda_val = simulation_stats[0]["Lambda"]

# Fitting distribusi
params_normal = stats.norm.fit(sample_data)
params_binom = (len(sample_data), np.mean(sample_data) / len(sample_data))  # Approx
# Estimate parameters for the negative binomial distribution using method of moments
mean_sample = np.mean(sample_data)
var_sample = np.var(sample_data)

# Calculate parameters r (number of successes) and p (probability of success)
if var_sample > mean_sample:
    r_nbinom = mean_sample**2 / (var_sample - mean_sample)
    p_nbinom = r_nbinom / (r_nbinom + mean_sample)
    params_nbinom_str = f"r={r_nbinom:.2f}, p={p_nbinom:.4f}"
else:
    params_nbinom_str = "N/A (var ≤ mean)"

comparison_dist = [
    ["Poisson", f"λ = {lambda_val:.2f}", f"{ks_poisson.statistic:.4f}", f"{ks_poisson.pvalue:.4f}",
     "Ya - Memodelkan jumlah kejadian dalam interval tetap (1 hari)"],
    ["Normal", f"μ = {params_normal[0]:.2f}, σ = {params_normal[1]:.2f}", 
     f"{ks_normal.statistic:.4f}", f"{ks_normal.pvalue:.4f}",
     "Tidak - Variabel kontinu, bisa menghasilkan nilai negatif atau non-integer"],
    ["Binomial", f"n={len(sample_data)}, p={np.mean(sample_data)/len(sample_data):.4f}", 
     "N/A", "N/A", 
     "Kurang cocok - Mengasumsikan jumlah percobaan tetap dengan probabilitas sukses konstan"],
    ["Negative Binomial", params_nbinom_str, "N/A", "N/A",
     "Mungkin cocok - Memodelkan jumlah sukses hingga r kegagalan terjadi"]
]

dist_headers = ["Distribusi", "Parameter", "KS Statistic", "p-value", "Kesesuaian untuk Kasus"]
print(tabulate(comparison_dist, headers=dist_headers, tablefmt="grid"))

# 9. Visualisasi perbandingan dengan distribusi lain sebagai gambar terpisah
plt.figure(figsize=(10, 6))

# Histogram data
sns.histplot(sample_data, stat="density", kde=False, label="Data Sampel", alpha=0.5)

# Plot PMF/PDF dari distribusi yang berbeda
x = np.arange(0, max(sample_data) + 5)
x_cont = np.linspace(0, max(sample_data) + 5, 100)

# Poisson PMF
plt.plot(x, stats.poisson.pmf(x, lambda_val), 'o-', 
         label=f'Poisson (λ={lambda_val:.2f})', alpha=0.7)

# Normal PDF
plt.plot(x_cont, stats.norm.pdf(x_cont, *params_normal), '--', 
         label=f'Normal (μ={params_normal[0]:.2f}, σ={params_normal[1]:.2f})', alpha=0.7)

# Binomial PMF (approximate)
n, p = 30, np.mean(sample_data) / 30  # Approx
plt.plot(x, stats.binom.pmf(x, n, p), '.-.', 
         label=f'Binomial (n={n}, p={p:.3f})', alpha=0.7)

plt.title("Perbandingan Distribusi untuk Data Jumlah Cacat")
plt.xlabel("Jumlah Barang Cacat")
plt.ylabel("Probabilitas / Densitas")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
# Save distribution comparison as separate image
plt.savefig('Image/perbandingan_distribusi_histogram.png', dpi=300, bbox_inches='tight')
plt.close()

# QQ Plot untuk Distribusi Poisson sebagai gambar terpisah
plt.figure(figsize=(10, 6))
stats.probplot(sample_data, dist=stats.poisson, sparams=(lambda_val,), plot=plt)
plt.title("QQ Plot untuk Distribusi Poisson")
plt.grid(alpha=0.3)
plt.tight_layout()
# Save QQ plot as separate image
plt.savefig('Image/qq_plot_poisson.png', dpi=300, bbox_inches='tight')
plt.close()

# Kesimpulan Analisis
print("\n===== Kesimpulan Analisis =====")
print("""
1. Distribusi Poisson menunjukkan kesesuaian yang baik dengan data simulasi jumlah barang cacat.
2. Parameter λ (lambda) merepresentasikan rata-rata jumlah cacat per hari.
3. Dalam praktik nyata, nilai lambda bisa bervariasi tergantung pada kompleksitas produk,
   ketahanan mesin, dan kualitas kontrol.
4. Distribusi normal kurang cocok karena bisa menghasilkan nilai negatif atau non-integer.
5. Distribusi binomial mengasumsikan jumlah percobaan tetap, yang mungkin tidak sesuai
   untuk kasus produksi dengan jumlah item yang sangat besar.
6. Jika frekuensi cacat bervariasi signifikan, distribusi negative binomial bisa jadi
   alternatif yang lebih baik karena memungkinkan overdispersion.
""")

===== Perbandingan Statistik Hasil Simulasi dengan Nilai Teoritis =====
+---------------+-----------------------+---------------------------+--------------------------+-------------------------+
| Simulasi      | Mean (Sim/Teoritis)   | Variance (Sim/Teoritis)   | Std Dev (Sim/Teoritis)   | P(X=0) (Sim/Teoritis)   |
+===============+=======================+===========================+==========================+=========================+
| Sim 1 (λ=4.0) | 3.93 / 4.00           | 3.06 / 4.00               | 1.75 / 2.00              | 0.0333 / 0.0183         |
+---------------+-----------------------+---------------------------+--------------------------+-------------------------+
| Sim 2 (λ=4.5) | 4.50 / 4.50           | 5.98 / 4.50               | 2.45 / 2.12              | 0.0000 / 0.0111         |
+---------------+-----------------------+---------------------------+--------------------------+-------------------------+
| Sim 3 (λ=5.0) | 4.47 / 5.00           | 3.78 / 5.00              